In [2]:

import json
from langchain_groq import ChatGroq

from pydantic import BaseModel, Field
from langchain_core.messages import (
    AIMessage, 
    HumanMessage,
    SystemMessage,
    BaseMessage
)
from typing import Optional, TypedDict, List, Dict, Any, Union
from langgraph.constants import Send

from Classes.state_classes import *

from Nodes.load_xml_instructions import load_xml_instructions

def build_hierarchy_string(filtered_metadata, parent_id=None, indent=0):
    """
    Convert multiple dimensions of `filtered_metadata` into a hierarchical string representation
    with parents below their children.
    """
    result = ""
    for metadata in filtered_metadata:
        dimension_content = metadata.get("dimensionContent", [])
        for item in dimension_content:
            item_parent_id = item.get("ParentID")
            if item_parent_id == {}:
                item_parent_id = None

            if item_parent_id == parent_id:
                # First, recursively add children
                result += build_hierarchy_string([metadata], parent_id=item["ID"], indent=indent + 1)
                # Then, add the current item's name with indentation
                result += "\t" * indent + f"{item['Name']}\n"
    return result

class FixedListReply(BaseModel):
    dimensions: list
    items: List[dict]

def create_fixed_list(state: ListSubchartState):
    current_list = state["List"]
    all_metadata = state["ReportMetadata"]
    chosen_dims = state.get("dimensions", [])

    filtered_metadata = []
    dims = []
    for dim in all_metadata:
        if dim["name"] in chosen_dims:
            new_dim = {
                "name": dim["name"],
                "alias": dim["alias"],
                "dimensionContent": dim.get("dimensionContent", [])
            }
            filtered_metadata.append(new_dim)
            dims.append(dim["name"])

    fixedlist_prompt = load_xml_instructions("fixedlist_prompt.xml")
    system_msg = SystemMessage(content=fixedlist_prompt)

    user_input = {
        "listObject": current_list,
        "dimensions": dims,
        "hierarchy": build_hierarchy_string(filtered_metadata)
    }
    user_msg = HumanMessage(content=json.dumps(user_input, indent=2))

    modelSpecLocal = ChatGroq(
        temperature=0.5,
        model_name="llama-3.3-70b-versatile",
        api_key="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",
        disable_streaming=True
    )

    structured_llm = modelSpecLocal.with_structured_output(
        FixedListReply,
        method="json_mode",
        include_raw=True
    )
    conversation = [system_msg, user_msg]
    output = structured_llm.invoke(conversation, stream=False, response_format="json")
    parsed_output = output["parsed"]
    final_list = parsed_output.model_dump()

    list_name = current_list.get("list", "Unnamed List")
    named_list = {list_name: final_list}

    return {"JsonLists": [named_list] }


import json


with open("sample_listsubchartstate.json", "r", encoding="utf-8") as f:
    stateobj = ListSubchartState(json.load(f))
    resultstate = create_fixed_list(stateobj)
    print(resultstate)

{'JsonLists': [{'ProfitAndLossAccounts': {'dimensions': ['Account'], 'items': [{'name': 'R1', 'dimensions': {'Account': {'member': 'Gross Sales'}}, 'displayName': '[Account]', 'format': '', 'drilldown': 'Account.[Account].descendants'}, {'name': 'R2', 'dimensions': {'Account': {'member': 'Net Sales'}}, 'displayName': '[Account]', 'format': 'bold_lineabove_skip'}, {'name': 'R3', 'dimensions': {'Account': {'member': 'Cost of goods sold'}}, 'displayName': '[Account]', 'format': ''}, {'name': 'R4', 'dimensions': {'Account': {'member': 'Gross Margin'}}, 'displayName': '[Account]', 'format': 'bold_skip_lineabove'}, {'name': 'R5', 'dimensions': {'Account': {'member': 'Staff Expenses'}}, 'displayName': '[Account]', 'format': ''}, {'name': 'R6', 'dimensions': {'Account': {'member': 'Rent & Utilities'}}, 'displayName': '[Account]', 'format': '', 'drilldown': 'Account.[Account].descendants'}, {'name': 'R7', 'dimensions': {'Account': {'member': 'Depreciation'}}, 'displayName': '[Account]', 'format